In [5]:
from shared.preface import *
import shared.functions as fct
import shared.my_units as my
import shared.control_center as CC

In [ ]:
def consumer(queue):
    while True:
        msg = queue.get()
        if (msg == 'DONE'):
            break

def producer(count, queue):
    for i in range(count):
        queue.put(i)
    queue.put('DONE')

queue = multiprocessing.Queue()
consumer_process = multiprocessing.Process(target=consumer, args=[queue])
consumer_process.daemon = True
consumer_process.start()

count = 10**4
producer(count, queue)
Send data to the queue


consumer_process.join()

print("Sent {0} numbers to Queue()".format(count))

In [12]:
from multiprocessing import Process, Queue, Pool, Manager
def writer(i,q):
    message = f'I am Process {i}'
    q.put(message)

def reader(i,q):
    message = q.get()
    print(message)

if __name__ == "__main__":

    # Create manager
    m = Manager()

    # Create multiprocessing queue
    q = m.Queue()

    # Create a group of parallel writers and start them
    for i in range(60000):
        Process(target=writer, args=(i,q,)).start()

    # Create multiprocessing pool
    p = Pool(1)

    # Create a group of parallel readers and start them
    # Number of readers is matching the number of writers
    # However, the number of simultaneously running
    # readers is constrained to the pool size

    readers = []
    for i in range(60000):
        readers.append(p.apply_async(reader, (i,q,)))

    # Wait for the asynchrounous reader threads to finish
    [r.get() for r in readers]

Process ForkPoolWorker-151:
Traceback (most recent call last):
  File "/home/fabian/anaconda3/envs/NC_numba/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/fabian/anaconda3/envs/NC_numba/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/fabian/anaconda3/envs/NC_numba/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home/fabian/anaconda3/envs/NC_numba/lib/python3.9/multiprocessing/queues.py", line 366, in get
    res = self._reader.recv_bytes()
  File "/home/fabian/anaconda3/envs/NC_numba/lib/python3.9/multiprocessing/connection.py", line 221, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/fabian/anaconda3/envs/NC_numba/lib/python3.9/multiprocessing/connection.py", line 419, in _recv_bytes
    buf = self._recv(4)
  File "/home/fabian/anaconda3/envs/NC_numba/lib/python3.9/multiprocessing/connection.py", line 384, in 

Process Process-7617:
Traceback (most recent call last):
  File "/home/fabian/anaconda3/envs/NC_numba/lib/python3.9/multiprocessing/process.py", line 299, in _bootstrap
    util._close_stdin()
  File "/home/fabian/anaconda3/envs/NC_numba/lib/python3.9/multiprocessing/util.py", line 422, in _close_stdin
    sys.stdin = open(fd, closefd=False)
  File "/home/fabian/anaconda3/envs/NC_numba/lib/python3.9/_bootlocale.py", line 33, in getpreferredencoding
    def getpreferredencoding(do_setlocale=True):
KeyboardInterrupt


KeyboardInterrupt: 

## Manual conversion factor for grav_pot function.

In [ ]:
units_raw = 1*const.G.unit * 1*my.rho0_NFW.unit * (1*unit.kpc)**2
units_goal = units_raw.to(unit.m**2/unit.s**2)

print(units_raw)
print(units_goal)

## Sorting logic for ndim arrays

In [ ]:
x = np.array([[4, 6, 5], [2, 1, 0], [1,6,5]])
y = np.array([[0, 1, 5], [2, 4, 0], [7,6,5]])

ind = x[:, 0].argsort()

x_sort = x[ind]
y_sort = y[ind]

print(x_sort)
print(y_sort)



# Various tests on individual functions

## Number integral

In [ ]:
FDarr = np.array([0.3,0.4,0.1,0.01])
p_arr = np.array([0.2, 0.1, 5., 8.]) * my.T_nu_eV.value

order = p_arr.argsort()
p_sort, FD_sort = p_arr[order], FDarr[order]
# print(p_sort, FD_sort)

n_int = np.trapz(p_sort**2 * FD_sort, p_sort)
print(n_int)

n_sum = np.sum(p_sort**2 * FD_sort)
print(n_sum)

## Integral for cosmic time

In [ ]:
def t_integrand_a(a):

    # original H0 in units ~[1/s], we only need the value
    H0 = my.H0.to(unit.s**-1).value

    a_dot = np.sqrt(my.Omega_m0/a**3 + my.Omega_L0)*H0*a
    t_int = 1/a_dot

    return t_int

t, err = quad(t_integrand_a, 0, 1)
t_uni, err_uni = (t*unit.s).to(unit.Gyr), (err*unit.s).to(unit.Gyr)
print(t_uni, err_uni)

In [ ]:
def t_integrand_z(z):

    # original H0 in units ~[1/s], we only need the value
    H0 = my.H0.to(unit.s**-1).value

    a_dot = np.sqrt(my.Omega_m0*(1+z)**3 + my.Omega_L0)*H0*(1+z)
    t_int = 1/a_dot

    return t_int

t, err = quad(t_integrand_z, 0, np.inf)
t_uni, err_uni = (t*unit.s).to(unit.Gyr), (err*unit.s).to(unit.Gyr)
print(t_uni, err_uni)

## Initial velocity limits

In [ ]:
m_sim_eV = 0.05*unit.eV
low_kpc, upp_kpc = fct.velocity_limits_of_m_nu(0.01, 10., m_sim_eV, mode='kpc/s')
print(low_kpc,upp_kpc)

m_sim_eV = 0.05*unit.eV
low_km, upp_km = fct.velocity_limits_of_m_nu(0.01, 10., m_sim_eV, mode='km/s')
print(low_km,upp_km)

## Simulated velocities in [kpc/s] to momenta in [eV] for any (neutrino) mass

In [ ]:
u_sim = np.load(f'neutrino_vectors/nu_2.npy')[-1,3:6]
m_sim_eV = 0.05*unit.eV
m_target_eV = 0.05*unit.eV

p, y = fct.u_to_p_eV(u_sim, m_sim_eV, m_target_eV)
print(p, y)

## 1/hc to cm^-1/eV

In [ ]:
hc_neg1 = (1/const.h/const.c).to(1/unit.cm/unit.eV)
print(hc_neg1)

n_cm3 = 1*unit.eV**3 * hc_neg1**3
print(n_cm3)

## Beta factor

In [ ]:
def c_vir_avg(z, M_vir):
    # Functions from Mertsch et al. (2020), eqns. (12) and (13) in ref. [40].
    a_of_z = 0.537 + (0.488)*np.exp(-0.718*np.power(z, 1.08))
    b_of_z = -0.097 + 0.024*z

    # Argument in log has to be dimensionless
    arg_in_log = (M_vir / (1.e12 / my.h * unit.M_sun)).value

    # Calculate avergae c_vir
    c_vir_avg = np.power(a_of_z + b_of_z*np.log10(arg_in_log), 10.)

    return c_vir_avg

# Beta is then obtained from c_vir_avg(0, M_vir) and c_vir(0, M_vir).
beta = (333.5/19.9) / c_vir_avg(0, my.Mvir_NFW)
print(beta)

## Table 1

In [ ]:
# Tests for c_vir and the beta coefficient

cis = fct.c_vir(0, my.Mvir_NFW)
cis1 = fct.c_vir(4, my.Mvir_NFW)
print(cis, cis1)

In [ ]:
z_test = 4
print('R_vir:', fct.R_vir(z_test, my.Mvir_NFW))
print('scale_radius:', fct.scale_radius(z_test, my.Mvir_NFW))

## Critical Density

In [ ]:
crit = fct.rho_crit(0)
print('Check if this matches critical density of universe today:')
print(crit)

## Unit Conversion Tests

In [ ]:
print(my.T_nu.to(unit.eV, unit.temperature_energy()))

## Derivative vector values

In [ ]:
z = 0
x_i = np.array([8.5,0.,0.])*unit.kpc

t2 = fct.dPsi_dxi_NFW(x_i, z, my.rho0_NFW, my.Mvir_NFW)
print(type(t2), t2)

## Time Variable s

In [ ]:
s = fct.s_of_z(4)
print('Value of time variable s in seconds at redhshift 4','\n', s)

h_s = my.H0.to(1/unit.s)
print('Age of universe in seconds:','\n', 1/h_s)

In [ ]:
s = fct.s_of_z(4)
print('Value of time variable s in seconds at redhshift 4','\n', s)

h_s = my.H0.to(1/unit.s)
print('Age of universe in seconds:','\n', 1/h_s)

## Initial Velocities

In [ ]:
from backtracing_numba import draw_ui

# Draw initial velocities.
ui = draw_ui(CC.PHIs, CC.THETAs, CC.Vs)*my.Uunit
ui = ui.to(unit.km/unit.s).value

ux = ui[:,0]
uy = ui[:,1]
uz = ui[:,2]

x = np.arange(len(ux))
y = np.arange(len(uy))
z = np.arange(len(uz))

plt.plot(x, ux, label='x-axis', )
plt.plot(y, uy, label='y-axis', )
plt.plot(z, uz, label='z-axis', alpha=0.5)
plt.title(f'Initial Velocities - {CC.NR_OF_NEUTRINOS} neutrinos')
plt.xlabel('Neutrino number')
plt.ylabel('Velocity w.r.t. axis in [km/s]')
plt.legend()
plt.savefig('check_plots/initial_velocities.pdf')

## Simpy derivative

## Redshift array for integration steps

In [ ]:
# OLD
'''
zeds_test = CC.ZEDS
print(len(zeds_test))
print(zeds_test[0], zeds_test[-1])

plt.scatter(zeds_test, zeds_test, s=1)
plt.show()
'''

Z_START, Z_STOP, Z_AMOUNT = 0., 4., 199
Z_START_LOG = 1e-1
zeds_pre = np.geomspace(Z_START_LOG, Z_STOP, Z_AMOUNT) - Z_START_LOG
ZEDS = np.insert(zeds_pre, len(zeds_pre), 4.)

z_early = np.linspace(0,2,200)
z_late = np.linspace(2.01,4,100)

zeds_test = np.concatenate((z_early, z_late))
print(len(zeds_test), zeds_test.shape)
print(zeds_test[0], zeds_test[-1])

plt.scatter(zeds_test, zeds_test, s=1)
plt.show()

## Faking double amount of particles to test integral

In [ ]:
# Read in neutrino vectors.
Ns = np.arange(CC.NR_OF_NEUTRINOS, dtype=int)  # Nr. of neutrinos
sim = np.array([np.load(f'neutrino_vectors/nu_{Nr+1}.npy') for Nr in Ns])

sim_fake = np.copy(sim)

for i, fake_arr in enumerate(sim_fake):
    np.save(f'neutrino_vectors/nu_{i+1+CC.NR_OF_NEUTRINOS}.npy', fake_arr)

## Shape of derivative for different redshifts

In [ ]:
z_arr = np.array([0.1,0.5,2,4.])
r = np.linspace(1,1000,100)*unit.kpc

fig, axs = plt.subplots(2,2, figsize=(12,10))
fig.suptitle('Shape of derivative', fontsize=18)

for j, z in enumerate(z_arr):

    # Compute values dependent on redshift.
    r_vir = fct.R_vir(z, my.Mvir_NFW)
    r_s = r_vir / fct.c_vir(z, my.Mvir_NFW)

    m = np.minimum(r, r_vir)
    M = np.maximum(r, r_vir)

    #! Ratios has to be unitless (e.g. else np.log yields 0.).
    ratio1 = (m/r_s).value
    ratio2 = (r/r_s).value
    ratio3 = (r_vir/M).value

    # Derivative in compact notation with m and M (without x_i).
    prefactor = 4.*np.pi*const.G*my.rho0_NFW*r_s**2./r**2.*(1*unit.kpc)
    term1 = np.log(1.+ratio1) / ratio2
    term2 = ratio3 / (1.+ratio1)
    derivative_vector = (prefactor * (term1 - term2)).to(unit.kpc/unit.s**2.)

    if np.any(derivative_vector.value) < 0.:
        print('neg. derivative!')

    i = 0
    if j in (2,3):
        i = 1
        j -= 2

    axs[i,j].plot(r, derivative_vector)
    axs[i,j].set_title(f'Redshift {z}')
    axs[i,j].set_xlabel('r')
    axs[i,j].set_ylabel('derivative')


## Fermi-Dirac distr. in Ringwald & Wong (2004)

In [ ]:
def Fermi_Dirac_Ringwald_and_Wong(p):
    # Plug into Fermi-Dirac distribution 
    arg_of_exp = (p/my.T_nu_eV).value
    f_of_p = expit(-arg_of_exp)

    return f_of_p

# Original Fermi-Dirac distr.
pOG = np.geomspace(0.01*my.T_nu_eV, 13*my.T_nu_eV, CC.NR_OF_NEUTRINOS)
FDvalsOG = Fermi_Dirac_Ringwald_and_Wong(pOG)
yOG = pOG/my.T_nu_eV.value

plt.loglog(yOG, FDvalsOG, label='Fermi-Dirac')
plt.title('Fermi-Dirac distr. of initial momentum')
plt.ylabel('f(y)')
plt.xlabel('y=p/T_nu')
plt.legend(loc='lower left')
plt.savefig('check_plots/FDdistr_RingwaldWong.pdf')